In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write

In [2]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from pyngrok import ngrok, conf

conf.get_default().auth_token = "2zg8dx2Dzix4o9HbGHbygDKWDYV_2RgojCTxdTzb7zogEV5qV"


In [ ]:
# Install dependencies (only once)
#!pip install -q fastapi uvicorn nest_asyncio pyngrok vllm transformers

# --- Imports ---
from fastapi import FastAPI, Request
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# --- Patch for Colab's event loop ---
nest_asyncio.apply()

# --- FastAPI App ---
app = FastAPI()

# Model to use (Gemma 2B Instruct)
model_id = "google/gemma-1.1-2b-it"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load vLLM model
llm = LLM(
    model=model_id,
    tensor_parallel_size=1,
    dtype="auto",  # Automatically selects FP16 on T4
    max_model_len=8192
)

# --- API Endpoint ---
@app.post("/generate")
async def generate(request: Request):
    data = await request.json()
    prompt_text = data["prompt"]

    # Apply chat-style formatting
    prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt_text}],
        add_generation_prompt=True,
        tokenize=False
    )

    sampling = SamplingParams(
        temperature=0.7,
        top_p=0.9,
        max_tokens=256
    )

    outputs = llm.generate(prompt, sampling)
    return {"response": outputs[0].outputs[0].text}

# --- Run server and expose with ngrok ---
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print(f"🚀 Public URL: {public_url}")

print(f"🚀 FastAPI server running at: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)


INFO 07-13 06:50:42 [__init__.py:244] Automatically detected platform cuda.
INFO 07-13 06:51:03 [config.py:841] This model supports multiple tasks: {'classify', 'embed', 'reward', 'generate'}. Defaulting to 'generate'.
WARNING 07-13 06:51:03 [config.py:3320] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 07-13 06:51:03 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 07-13 06:51:03 [config.py:1472] Using max model len 8192
WARNING 07-13 06:51:03 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 07-13 06:51:03 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='google/gemma-1.1-2b-it', speculative_config=None, tokenizer='google/gemma-1.1-2b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.floa

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 07-13 06:51:29 [default_loader.py:272] Loading weights took 22.07 seconds
INFO 07-13 06:51:30 [model_runner.py:1203] Model loading took 4.6721 GiB and 22.779751 seconds
INFO 07-13 06:51:37 [worker.py:294] Memory profiling takes 6.63 seconds
INFO 07-13 06:51:37 [worker.py:294] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 07-13 06:51:37 [worker.py:294] model weights take 4.67GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 2.36GiB; the rest of the memory reserved for KV Cache is 6.21GiB.
INFO 07-13 06:51:38 [executor_base.py:113] # cuda blocks: 22606, # CPU blocks: 14563
INFO 07-13 06:51:38 [executor_base.py:118] Maximum concurrency for 8192 tokens per request: 44.15x
INFO 07-13 06:51:41 [model_runner.py:1513] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' i

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 07-13 06:52:51 [model_runner.py:1671] Graph capturing finished in 70 secs, took 0.17 GiB
INFO 07-13 06:52:51 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 80.49 seconds
🚀 Public URL: NgrokTunnel: "https://df2a88d5ef78.ngrok-free.app" -> "http://localhost:8000"
🚀 FastAPI server running at: NgrokTunnel: "https://df2a88d5ef78.ngrok-free.app" -> "http://localhost:8000"


...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

INFO:     2401:4900:1cc4:ccbe:2044:9d2c:8935:7ae2:0 - "POST /generate HTTP/1.1" 200 OK


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

INFO:     2401:4900:1cc4:ccbe:2044:9d2c:8935:7ae2:0 - "POST /generate HTTP/1.1" 200 OK
